# Using Plotly graphs with TINC

In [ ]:
import tinc
from numpy import *

In [ ]:
p = tinc.Parameter("param", "", 0, 10, 1.5)
p.values = linspace(0, 10, 21)

In [ ]:
p.values

In [ ]:
p2 = tinc.Parameter("param2", "", 0, 10, 2.5)
p2.values = linspace(0, 10, 21)

In [ ]:
from ipywidgets import VBox

Create a plotly ```FigureWidget``` and set the data to contain the plot you need:

In [ ]:
import plotly.graph_objects as go
x = linspace(0, 100, 101)
y = linspace(0,100, 101)
g = go.FigureWidget(data=[go.Scatter(x = x, y =y )])

Write a callback function that gets the ```FigureWidget``` global object and change the data within the data objects for it:

In [ ]:
def update_graph(ignored_value = 0):
    global g
    line = p.value * x * x + p2.value * x *50
    g.data[0].y = line

In [ ]:
p.register_callback(update_graph)
p2.register_callback(update_graph)

VBox([p.interactive_control(), p2.interactive_control(), g])

In [ ]:
# clear the callbacks as we will be doing big changes
p.clear_callbacks()
p2.clear_callbacks()

# 3D plot

The only caveat when using TINC with plotly in this way is that you must initialize the ```FigureWidget``` with the same type an number of graph objects you plan to use. For example, to use 

In [ ]:
x = linspace(0, 100, 101)
line = p.value * x * x + p2.value * x *50
g = go.FigureWidget(data=[go.Mesh3d(x=x,
                   y=line,
                   z=-line + x,
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  ),go.Mesh3d(x=x,
                   y=line,
                   z=-line + x,
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

def update_graph(ignored_value = 0):
    global g
    x = linspace(0, 100, 101)
    if p.value > 3:
        line = p.value * x * x + p2.value * x *50
        g.update(data = [go.Mesh3d(x=x,
                   y=line,
                   z=-line + x,
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])
    else:
        line = p.value * x * x + p2.value * x *50
        g.update(data = [go.Mesh3d(x=x,
                   y=line,
                   z=-line + x,
                   opacity=0.5,
                   color='rgba(22,244,100,0.6)'
                  ),go.Mesh3d(x=x,
                   y=line + x + 100,
                   z=-line + x,
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])
        print("hello")
        

In [ ]:
p.register_callback(update_graph)
p2.register_callback(update_graph)

VBox([p.interactive_control(), p2.interactive_control(), g])

## Using cache for plotly plots

In [ ]:
import time

def generate_data(param, param2):
    time.sleep(2.5) # Process takes time
    x = linspace(0, 100, 101)
    return param * x * x + param2 * x *50

In [ ]:
ps = tinc.ParameterSpace('ps')

ps.register_dimensions([p, p2])

In [ ]:
ps.run_process(generate_data)

In [ ]:
ps.enable_cache()

In [ ]:
ps.run_process(generate_data)

In [ ]:
import plotly.graph_objects as go
x = linspace(0, 100, 101)
y = linspace(0,100, 101)
g = go.FigureWidget(data=[go.Scatter(x = x, y =y )])

def update_graph(ignored_value = 0):
    global g
    g.data[0]['line']['color'] = "#000000" # Display black line while computation takes place
    line = ps.run_process(generate_data)
    g.data[0]['line']['color'] = "#3333ff" # Set color when done
    g.data[0].y = line

p.clear_callbacks()
p2.clear_callbacks()

p.register_callback(update_graph)
p2.register_callback(update_graph)

VBox([p.interactive_control(), p2.interactive_control(), g])

In [ ]:
g.data[0].